# Separate Semi-Supervised Training Algorithms

## 1. Pseudo-Labeling Algorithm

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

class PseudoLabelingTrainer:
    """Entraîneur pour pseudo-labeling"""
    
    def __init__(self, data_dir='preprocessed_ml_data'):
        self.data_dir = data_dir
        self.classes = ['dark', 'light', 'mid-dark', 'mid-light']
        self.load_data()
        self.results = []
    
    def load_data(self):
        """Charge les données prétraitées"""
        print("📂 Chargement des données prétraitées...")
        
        # Charger les datasets
        datasets = {}
        for split in ['train', 'val', 'test', 'unlabelled']:
            data = np.load(os.path.join(self.data_dir, f'{split}_data.npz'))
            datasets[split] = {
                'X': data['X'],
                'y': data['y'],
                'filenames': data['filenames']
            }
        
        self.X_train = datasets['train']['X']
        self.y_train = datasets['train']['y']
        self.X_val = datasets['val']['X'] 
        self.y_val = datasets['val']['y']
        self.X_test = datasets['test']['X']
        self.y_test = datasets['test']['y']
        self.X_unlabelled = datasets['unlabelled']['X']
        
        print(f"  ✓ Données chargées: {len(self.X_train)} train, {len(self.X_val)} val, {len(self.X_test)} test, {len(self.X_unlabelled)} unlabelled")
    
    def train_pseudo_labeling(self, base_classifier='rf', confidence_threshold=0.8, max_iterations=10):
        """Implémente le pseudo-labeling"""
        print(f"\n🔄 PSEUDO-LABELING avec {base_classifier}, seuil={confidence_threshold}")
        
        # Sélectionner le classifieur de base
        if base_classifier == 'rf':
            clf = RandomForestClassifier(n_estimators=100, random_state=42)
        elif base_classifier == 'svm':
            clf = SVC(probability=True, random_state=42)
        elif base_classifier == 'lr':
            clf = LogisticRegression(random_state=42, max_iter=1000)
        
        # Données initiales
        X_labeled = self.X_train.copy()
        y_labeled = self.y_train.copy()
        X_unlabeled = self.X_unlabelled.copy()
        
        best_accuracy = 0
        iteration_results = []
        
        for iteration in range(max_iterations):
            print(f"  Itération {iteration + 1}/{max_iterations}")
            
            # Entraîner sur données labellisées actuelles
            clf.fit(X_labeled, y_labeled)
            
            # Prédire sur données non-labellisées
            if hasattr(clf, 'predict_proba'):
                proba_predictions = clf.predict_proba(X_unlabeled)
                predictions = np.argmax(proba_predictions, axis=1)
                confidences = np.max(proba_predictions, axis=1)
            else:
                predictions = clf.predict(X_unlabeled)
                confidences = np.ones(len(predictions))  # Pas de probabilités
            
            # Sélectionner les prédictions confiantes
            confident_mask = confidences >= confidence_threshold
            X_confident = X_unlabeled[confident_mask]
            y_confident = predictions[confident_mask]
            
            if len(X_confident) == 0:
                print(f"    Aucune prédiction confiante trouvée, arrêt à l'itération {iteration + 1}")
                break
            
            # Ajouter les données pseudo-labellisées
            X_labeled = np.vstack([X_labeled, X_confident])
            y_labeled = np.hstack([y_labeled, y_confident])
            
            # Retirer des données non-labellisées
            X_unlabeled = X_unlabeled[~confident_mask]
            
            print(f"    Ajouté {len(X_confident)} pseudo-labels, {len(X_unlabeled)} restants")
            
            # Évaluer sur validation
            val_accuracy = clf.score(self.X_val, self.y_val)
            iteration_results.append({
                'iteration': iteration + 1,
                'val_accuracy': val_accuracy,
                'pseudo_labels_added': len(X_confident),
                'remaining_unlabeled': len(X_unlabeled)
            })
            
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
        
        # Évaluation finale sur test
        test_accuracy = clf.score(self.X_test, self.y_test)
        test_f1 = f1_score(self.y_test, clf.predict(self.X_test), average='weighted')
        
        result = {
            'method': 'pseudo_labeling',
            'base_classifier': base_classifier,
            'confidence_threshold': confidence_threshold,
            'max_iterations': max_iterations,
            'final_test_accuracy': test_accuracy,
            'final_test_f1': test_f1,
            'best_val_accuracy': best_accuracy,
            'iterations': iteration_results,
            'total_pseudo_labels': len(X_labeled) - len(self.X_train)
        }
        
        self.results.append(result)
        print(f"  ✅ Test accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}")
        
        return result
    
    def run_multiple_sessions(self):
        """Exécute plusieurs sessions avec différents paramètres"""
        print("🚀 PSEUDO-LABELING: SESSIONS MULTIPLES")
        print("=" * 50)
        
        for clf in ['rf', 'svm', 'lr']:
            for threshold in [0.7, 0.8, 0.9]:
                self.train_pseudo_labeling(base_classifier=clf, confidence_threshold=threshold)
    
    def save_results(self, output_file='pseudo_labeling_results.pkl'):
        """Sauvegarde les résultats"""
        with open(output_file, 'wb') as f:
            pickle.dump(self.results, f)
        print(f"\n💾 Résultats sauvegardés dans {output_file}")
    
    def display_summary(self):
        """Affiche un résumé"""
        print("\n📊 RÉSUMÉ PSEUDO-LABELING")
        print("=" * 30)
        
        if not self.results:
            print("Aucun résultat")
            return
        
        df = pd.DataFrame(self.results)
        print(f"  Expériences: {len(df)}")
        print(f"  Accuracy moyenne: {df['final_test_accuracy'].mean():.4f}")
        print(f"  Meilleur résultat: {df['final_test_accuracy'].max():.4f}")
        
        # Meilleur résultat
        best_idx = df['final_test_accuracy'].idxmax()
        best = df.loc[best_idx]
        print(f"  Meilleure config: {best['base_classifier']} + seuil {best['confidence_threshold']}")

def main():
    """Fonction principale pour pseudo-labeling"""
    print("🎯 PSEUDO-LABELING POUR CLASSIFICATION DE TEINT DE PEAU")
    print("=" * 60)
    
    trainer = PseudoLabelingTrainer(data_dir='preprocessed_ml_data')
    trainer.run_multiple_sessions()
    trainer.save_results('pseudo_labeling_results.pkl')
    trainer.display_summary()
    
    print("\n✅ Pseudo-labeling terminé!")

if __name__ == "__main__":
    main()

🎯 PSEUDO-LABELING POUR CLASSIFICATION DE TEINT DE PEAU
📂 Chargement des données prétraitées...
  ✓ Données chargées: 5748 train, 1228 val, 1236 test, 19164 unlabelled
🚀 PSEUDO-LABELING: SESSIONS MULTIPLES

🔄 PSEUDO-LABELING avec rf, seuil=0.7
  Itération 1/10
    Ajouté 2402 pseudo-labels, 16762 restants
  Itération 2/10
    Ajouté 1172 pseudo-labels, 15590 restants
  Itération 3/10
    Ajouté 594 pseudo-labels, 14996 restants
  Itération 4/10
    Ajouté 358 pseudo-labels, 14638 restants
  Itération 5/10
    Ajouté 277 pseudo-labels, 14361 restants
  Itération 6/10
    Ajouté 201 pseudo-labels, 14160 restants
  Itération 7/10
    Ajouté 150 pseudo-labels, 14010 restants
  Itération 8/10
    Ajouté 112 pseudo-labels, 13898 restants
  Itération 9/10
    Ajouté 106 pseudo-labels, 13792 restants
  Itération 10/10
    Ajouté 83 pseudo-labels, 13709 restants
  ✅ Test accuracy: 0.6675, F1: 0.6569

🔄 PSEUDO-LABELING avec rf, seuil=0.8
  Itération 1/10
    Ajouté 679 pseudo-labels, 18485 restan

## 2. Graph-Based Methods Algorithm


In [4]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import time
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

class GraphBasedTrainer:
    """Entraîneur pour méthodes graph-based"""
    
    def __init__(self, data_dir='preprocessed_ml_data'):
        self.data_dir = data_dir
        self.classes = ['dark', 'light', 'mid-dark', 'mid-light']
        self.load_data()
        self.results = []
    
    def load_data(self):
        """Charge les données prétraitées"""
        print("📂 Chargement des données prétraitées...")
        
        datasets = {}
        for split in ['train', 'val', 'test', 'unlabelled']:
            data = np.load(os.path.join(self.data_dir, f'{split}_data.npz'))
            datasets[split] = {
                'X': data['X'],
                'y': data['y'],
                'filenames': data['filenames']
            }
        
        self.X_train = datasets['train']['X']
        self.y_train = datasets['train']['y']
        self.X_val = datasets['val']['X'] 
        self.y_val = datasets['val']['y']
        self.X_test = datasets['test']['X']
        self.y_test = datasets['test']['y']
        self.X_unlabelled = datasets['unlabelled']['X']
        
        print(f"  ✓ Données chargées: {len(self.X_train)} train, {len(self.X_val)} val, {len(self.X_test)} test, {len(self.X_unlabelled)} unlabelled")
    
    def train_graph_method(self, method='propagation', kernel='rbf', gamma=20, alpha=0.2):
        """Implémente les méthodes graph-based"""
        print(f"\n🕸️  {method.upper()}: kernel={kernel}, gamma={gamma}", end="")
        if method == 'spreading':
            print(f", alpha={alpha}")
        else:
            print("")
        
        # Combiner données labellisées et non-labellisées
        X_combined = np.vstack([self.X_train, self.X_unlabelled])
        y_combined = np.hstack([self.y_train, np.full(len(self.X_unlabelled), -1)])  # -1 pour non-labellisé
        
        # Sélectionner la méthode
        if method == 'propagation':
            clf = LabelPropagation(kernel=kernel, gamma=gamma, max_iter=1000)
        elif method == 'spreading':
            clf = LabelSpreading(kernel=kernel, gamma=gamma, alpha=alpha, max_iter=1000)
        
        # Entraîner
        start_time = time.time()
        clf.fit(X_combined, y_combined)
        training_time = time.time() - start_time
        
        # Prédire sur test
        y_pred = clf.predict(self.X_test)
        
        # Évaluer
        test_accuracy = accuracy_score(self.y_test, y_pred)
        test_f1 = f1_score(self.y_test, y_pred, average='weighted')
        
        result = {
            'method': f'graph_{method}',
            'kernel': kernel,
            'gamma': gamma,
            'alpha': alpha if method == 'spreading' else None,
            'test_accuracy': test_accuracy,
            'test_f1': test_f1,
            'training_time': training_time
        }
        
        self.results.append(result)
        print(f"  ✅ Test accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}, Time: {training_time:.2f}s")
        
        return result
    
    def run_multiple_sessions(self):
        """Exécute plusieurs sessions avec différents paramètres"""
        print("🚀 GRAPH-BASED METHODS: SESSIONS MULTIPLES")
        print("=" * 50)
        
        for method in ['propagation', 'spreading']:
            for gamma in [10, 20, 50]:
                if method == 'propagation':
                    # LabelPropagation n'a pas de paramètre alpha
                    self.train_graph_method(method=method, gamma=gamma)
                else:
                    # LabelSpreading a un paramètre alpha
                    for alpha in [0.1, 0.2, 0.5]:
                        self.train_graph_method(method=method, gamma=gamma, alpha=alpha)
    
    def save_results(self, output_file='graph_based_results.pkl'):
        """Sauvegarde les résultats"""
        with open(output_file, 'wb') as f:
            pickle.dump(self.results, f)
        print(f"\n💾 Résultats sauvegardés dans {output_file}")
    
    def display_summary(self):
        """Affiche un résumé"""
        print("\n📊 RÉSUMÉ GRAPH-BASED METHODS")
        print("=" * 30)
        
        if not self.results:
            print("Aucun résultat")
            return
        
        df = pd.DataFrame(self.results)
        print(f"  Expériences: {len(df)}")
        print(f"  Accuracy moyenne: {df['test_accuracy'].mean():.4f}")
        print(f"  Meilleur résultat: {df['test_accuracy'].max():.4f}")
        
        # Meilleur résultat
        best_idx = df['test_accuracy'].idxmax()
        best = df.loc[best_idx]
        config_str = f"{best['method']} + gamma {best['gamma']}"
        if best['alpha'] is not None:
            config_str += f" + alpha {best['alpha']}"
        print(f"  Meilleure config: {config_str}")

def main():
    """Fonction principale pour graph-based methods"""
    print("🎯 GRAPH-BASED METHODS POUR CLASSIFICATION DE TEINT DE PEAU")
    print("=" * 60)
    
    trainer = GraphBasedTrainer(data_dir='preprocessed_ml_data')
    trainer.run_multiple_sessions()
    trainer.save_results('graph_based_results.pkl')
    trainer.display_summary()
    
    print("\n✅ Graph-based methods terminé!")

if __name__ == "__main__":
    main()

🎯 GRAPH-BASED METHODS POUR CLASSIFICATION DE TEINT DE PEAU
📂 Chargement des données prétraitées...
  ✓ Données chargées: 5748 train, 1228 val, 1236 test, 19164 unlabelled
🚀 GRAPH-BASED METHODS: SESSIONS MULTIPLES

🕸️  PROPAGATION: kernel=rbf, gamma=10
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 34.67s

🕸️  PROPAGATION: kernel=rbf, gamma=20
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 23.45s

🕸️  PROPAGATION: kernel=rbf, gamma=50
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 22.56s

🕸️  SPREADING: kernel=rbf, gamma=10, alpha=0.1
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 70.25s

🕸️  SPREADING: kernel=rbf, gamma=10, alpha=0.2
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 92.23s

🕸️  SPREADING: kernel=rbf, gamma=10, alpha=0.5
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 218.44s

🕸️  SPREADING: kernel=rbf, gamma=20, alpha=0.1
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 70.38s

🕸️  SPREADING: kernel=rbf, gamma=20, alpha=0.2
  ✅ Test accuracy: 0.2702, F1: 0.1393, Time: 105.88s

🕸️  SPREADI

## 3. Consistency Regularization Algorithm

In [3]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import time
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

class ConsistencyTrainer:
    """Entraîneur pour régularisation de cohérence"""
    
    def __init__(self, data_dir='preprocessed_ml_data'):
        self.data_dir = data_dir
        self.classes = ['dark', 'light', 'mid-dark', 'mid-light']
        self.load_data()
        self.results = []
    
    def load_data(self):
        """Charge les données prétraitées"""
        print("📂 Chargement des données prétraitées...")
        
        datasets = {}
        for split in ['train', 'val', 'test', 'unlabelled']:
            data = np.load(os.path.join(self.data_dir, f'{split}_data.npz'))
            datasets[split] = {
                'X': data['X'],
                'y': data['y'],
                'filenames': data['filenames']
            }
        
        self.X_train = datasets['train']['X']
        self.y_train = datasets['train']['y']
        self.X_val = datasets['val']['X'] 
        self.y_val = datasets['val']['y']
        self.X_test = datasets['test']['X']
        self.y_test = datasets['test']['y']
        self.X_unlabelled = datasets['unlabelled']['X']
        
        print(f"  ✓ Données chargées: {len(self.X_train)} train, {len(self.X_val)} val, {len(self.X_test)} test, {len(self.X_unlabelled)} unlabelled")
    
    def train_consistency_regularization(self, n_estimators=10, perturbation_strength=0.1):
        """Implémente la régularisation de cohérence avec ensemble"""
        print(f"\n🔄 CONSISTENCY REGULARIZATION: {n_estimators} estimators, perturbation={perturbation_strength}")
        
        # Créer des perturbations des données d'entraînement
        X_perturbed = []
        y_perturbed = []
        
        for _ in range(n_estimators):
            # Ajouter du bruit gaussien
            noise = np.random.normal(0, perturbation_strength, self.X_train.shape)
            X_pert = self.X_train + noise
            X_perturbed.append(X_pert)
            y_perturbed.append(self.y_train)
        
        X_perturbed = np.vstack(X_perturbed)
        y_perturbed = np.hstack(y_perturbed)
        
        # Entraîner un classifieur d'ensemble
        clf = ExtraTreesClassifier(n_estimators=n_estimators, random_state=42)
        
        start_time = time.time()
        clf.fit(X_perturbed, y_perturbed)
        training_time = time.time() - start_time
        
        # Évaluer
        y_pred = clf.predict(self.X_test)
        test_accuracy = accuracy_score(self.y_test, y_pred)
        test_f1 = f1_score(self.y_test, y_pred, average='weighted')
        
        result = {
            'method': 'consistency_regularization',
            'n_estimators': n_estimators,
            'perturbation_strength': perturbation_strength,
            'test_accuracy': test_accuracy,
            'test_f1': test_f1,
            'training_time': training_time
        }
        
        self.results.append(result)
        print(f"  ✅ Test accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}, Time: {training_time:.2f}s")
        
        return result
    
    def run_multiple_sessions(self):
        """Exécute plusieurs sessions avec différents paramètres"""
        print("🚀 CONSISTENCY REGULARIZATION: SESSIONS MULTIPLES")
        print("=" * 50)
        
        for n_est in [5, 10, 20]:
            for pert in [0.05, 0.1, 0.2]:
                self.train_consistency_regularization(n_estimators=n_est, perturbation_strength=pert)
    
    def save_results(self, output_file='consistency_results.pkl'):
        """Sauvegarde les résultats"""
        with open(output_file, 'wb') as f:
            pickle.dump(self.results, f)
        print(f"\n💾 Résultats sauvegardés dans {output_file}")
    
    def display_summary(self):
        """Affiche un résumé"""
        print("\n📊 RÉSUMÉ CONSISTENCY REGULARIZATION")
        print("=" * 35)
        
        if not self.results:
            print("Aucun résultat")
            return
        
        df = pd.DataFrame(self.results)
        print(f"  Expériences: {len(df)}")
        print(f"  Accuracy moyenne: {df['test_accuracy'].mean():.4f}")
        print(f"  Meilleur résultat: {df['test_accuracy'].max():.4f}")
        
        # Meilleur résultat
        best_idx = df['test_accuracy'].idxmax()
        best = df.loc[best_idx]
        print(f"  Meilleure config: {best['n_estimators']} estimators + perturbation {best['perturbation_strength']}")

def main():
    """Fonction principale pour consistency regularization"""
    print("🎯 CONSISTENCY REGULARIZATION POUR CLASSIFICATION DE TEINT DE PEAU")
    print("=" * 60)
    
    trainer = ConsistencyTrainer(data_dir='preprocessed_ml_data')
    trainer.run_multiple_sessions()
    trainer.save_results('consistency_results.pkl')
    trainer.display_summary()
    
    print("\n✅ Consistency regularization terminé!")

if __name__ == "__main__":
    main()

🎯 CONSISTENCY REGULARIZATION POUR CLASSIFICATION DE TEINT DE PEAU
📂 Chargement des données prétraitées...
  ✓ Données chargées: 5748 train, 1228 val, 1236 test, 19164 unlabelled
🚀 CONSISTENCY REGULARIZATION: SESSIONS MULTIPLES

🔄 CONSISTENCY REGULARIZATION: 5 estimators, perturbation=0.05
  ✅ Test accuracy: 0.5631, F1: 0.5494, Time: 0.31s

🔄 CONSISTENCY REGULARIZATION: 5 estimators, perturbation=0.1
  ✅ Test accuracy: 0.5761, F1: 0.5614, Time: 0.33s

🔄 CONSISTENCY REGULARIZATION: 5 estimators, perturbation=0.2
  ✅ Test accuracy: 0.5655, F1: 0.5477, Time: 0.34s

🔄 CONSISTENCY REGULARIZATION: 10 estimators, perturbation=0.05
  ✅ Test accuracy: 0.5979, F1: 0.5865, Time: 1.41s

🔄 CONSISTENCY REGULARIZATION: 10 estimators, perturbation=0.1
  ✅ Test accuracy: 0.5906, F1: 0.5776, Time: 1.33s

🔄 CONSISTENCY REGULARIZATION: 10 estimators, perturbation=0.2
  ✅ Test accuracy: 0.6303, F1: 0.6187, Time: 1.33s

🔄 CONSISTENCY REGULARIZATION: 20 estimators, perturbation=0.05
  ✅ Test accuracy: 0.6424,

# save the best model 

In [6]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import time
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

class ModelSelector:
    """Sélecteur et sauvegardeur du meilleur modèle"""
    
    def __init__(self, data_dir='preprocessed_ml_data'):
        self.data_dir = data_dir
        self.load_data()
    
    def load_data(self):
        """Charge les données prétraitées"""
        print("📂 Chargement des données prétraitées...")
        
        datasets = {}
        for split in ['train', 'val', 'test', 'unlabelled']:
            data = np.load(os.path.join(self.data_dir, f'{split}_data.npz'))
            datasets[split] = {
                'X': data['X'],
                'y': data['y'],
                'filenames': data['filenames']
            }
        
        self.X_train = datasets['train']['X']
        self.y_train = datasets['train']['y']
        self.X_val = datasets['val']['X'] 
        self.y_val = datasets['val']['y']
        self.X_test = datasets['test']['X']
        self.y_test = datasets['test']['y']
        self.X_unlabelled = datasets['unlabelled']['X']
        
        print(f"  ✓ Données chargées: {len(self.X_train)} train, {len(self.X_val)} val, {len(self.X_test)} test, {len(self.X_unlabelled)} unlabelled")
    
    def load_and_compare_results(self):
        """Charge et compare tous les résultats d'entraînement"""
        result_files = [
            'pseudo_labeling_results.pkl',
            'graph_based_results.pkl', 
            'consistency_results.pkl'
        ]
        
        all_results = []
        
        for file in result_files:
            if os.path.exists(file):
                with open(file, 'rb') as f:
                    results = pickle.load(f)
                    all_results.extend(results)
                    print(f"✓ Chargé {len(results)} résultats depuis {file}")
            else:
                print(f"⚠️ Fichier {file} non trouvé")
        
        if not all_results:
            print("❌ Aucun résultat trouvé!")
            return None
        
        return all_results
    
    def select_best_model(self, all_results):
        """Sélectionne le meilleur modèle basé sur l'accuracy de test"""
        # Fonction pour obtenir l'accuracy (gère les clés différentes)
        def get_accuracy(result):
            if 'test_accuracy' in result:
                return result['test_accuracy']
            elif 'final_test_accuracy' in result:
                return result['final_test_accuracy']
            else:
                return 0  # Valeur par défaut si pas trouvé
        
        best_result = max(all_results, key=get_accuracy)
        
        # Obtenir l'accuracy pour l'affichage
        accuracy = get_accuracy(best_result)
        f1_score_val = best_result.get('test_f1', best_result.get('final_test_f1', 0))
        
        print("\n🏆 MEILLEUR MODÈLE TROUVÉ:")
        print(f"  Méthode: {best_result['method']}")
        print(f"  Accuracy de test: {accuracy:.4f}")
        print(f"  F1-score: {f1_score_val:.4f}")
        
        # Afficher les paramètres spécifiques
        params = {k: v for k, v in best_result.items() 
                 if k not in ['method', 'test_accuracy', 'final_test_accuracy', 'test_f1', 'final_test_f1', 'training_time', 'iterations']}
        print(f"  Paramètres: {params}")
        
        return best_result
    
    def retrain_best_model(self, best_result):
        """Réentraîne le meilleur modèle avec tous les paramètres optimaux"""
        method = best_result['method']
        print(f"\n🔄 Réentraînement du modèle: {method}")
        
        if method.startswith('pseudo_labeling'):
            # Pseudo-labeling
            base_classifier = best_result['base_classifier']
            confidence_threshold = best_result['confidence_threshold']
            
            if base_classifier == 'rf':
                clf = RandomForestClassifier(n_estimators=100, random_state=42)
            elif base_classifier == 'svm':
                clf = SVC(probability=True, random_state=42)
            elif base_classifier == 'lr':
                clf = LogisticRegression(random_state=42, max_iter=1000)
            
            # Entraîner sur les données labellisées
            clf.fit(self.X_train, self.y_train)
            
        elif method.startswith('graph_'):
            # Graph-based
            method_type = method.split('_')[1]  # 'propagation' or 'spreading'
            kernel = best_result['kernel']
            gamma = best_result['gamma']
            
            X_combined = np.vstack([self.X_train, self.X_unlabelled])
            y_combined = np.hstack([self.y_train, np.full(len(self.X_unlabelled), -1)])
            
            if method_type == 'propagation':
                clf = LabelPropagation(kernel=kernel, gamma=gamma, max_iter=1000)
            elif method_type == 'spreading':
                alpha = best_result.get('alpha', 0.2)
                clf = LabelSpreading(kernel=kernel, gamma=gamma, alpha=alpha, max_iter=1000)
            
            clf.fit(X_combined, y_combined)
            
        elif method == 'consistency_regularization':
            # Consistency regularization
            n_estimators = best_result['n_estimators']
            perturbation_strength = best_result['perturbation_strength']
            
            # Créer des perturbations
            X_perturbed = []
            y_perturbed = []
            
            for _ in range(n_estimators):
                noise = np.random.normal(0, perturbation_strength, self.X_train.shape)
                X_pert = self.X_train + noise
                X_perturbed.append(X_pert)
                y_perturbed.append(self.y_train)
            
            X_perturbed = np.vstack(X_perturbed)
            y_perturbed = np.hstack(y_perturbed)
            
            clf = ExtraTreesClassifier(n_estimators=n_estimators, random_state=42)
            clf.fit(X_perturbed, y_perturbed)
        
        # Validation finale
        test_accuracy = clf.score(self.X_test, self.y_test)
        test_f1 = f1_score(self.y_test, clf.predict(self.X_test), average='weighted')
        
        print(f"  ✅ Modèle réentraîné - Test accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}")
        
        return clf
    
    def save_best_model(self, model, filename='best_model.pkl'):
        """Sauvegarde le meilleur modèle"""
        with open(filename, 'wb') as f:
            pickle.dump(model, f)
        print(f"💾 Modèle sauvegardé dans {filename}")

def main():
    """Fonction principale pour sélection et sauvegarde du meilleur modèle"""
    print("🎯 SÉLECTION ET SAUVEGARDE DU MEILLEUR MODÈLE")
    print("=" * 60)
    
    # Initialiser le sélecteur
    selector = ModelSelector(data_dir='preprocessed_ml_data')
    
    # Charger et comparer les résultats
    all_results = selector.load_and_compare_results()
    
    if all_results:
        # Sélectionner le meilleur
        best_result = selector.select_best_model(all_results)
        
        # Réentraîner le meilleur modèle
        best_model = selector.retrain_best_model(best_result)
        
        # Sauvegarder
        selector.save_best_model(best_model, 'best_model.pkl')
        
        print("\n✅ Meilleur modèle prêt pour le déploiement!")
        print("Vous pouvez maintenant lancer l'application Streamlit.")
    
    else:
        print("❌ Impossible de trouver des résultats d'entraînement.")
        print("Assurez-vous d'avoir exécuté les scripts d'entraînement.")

if __name__ == "__main__":
    main()

🎯 SÉLECTION ET SAUVEGARDE DU MEILLEUR MODÈLE
📂 Chargement des données prétraitées...
  ✓ Données chargées: 5748 train, 1228 val, 1236 test, 19164 unlabelled
✓ Chargé 9 résultats depuis pseudo_labeling_results.pkl
✓ Chargé 12 résultats depuis graph_based_results.pkl
✓ Chargé 9 résultats depuis consistency_results.pkl

🏆 MEILLEUR MODÈLE TROUVÉ:
  Méthode: pseudo_labeling
  Accuracy de test: 0.7362
  F1-score: 0.7341
  Paramètres: {'base_classifier': 'svm', 'confidence_threshold': 0.7, 'max_iterations': 10, 'best_val_accuracy': 0.738599348534202, 'total_pseudo_labels': 17088}

🔄 Réentraînement du modèle: pseudo_labeling
  ✅ Modèle réentraîné - Test accuracy: 0.7257, F1: 0.7250
💾 Modèle sauvegardé dans best_model.pkl

✅ Meilleur modèle prêt pour le déploiement!
Vous pouvez maintenant lancer l'application Streamlit.
